<center><h1>Titanic - Machine Learning from Disaster</h1></center>

In [423]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

In [424]:
#ignoring unessary warning
import warnings
warnings.filterwarnings('ignore')

In [425]:
#importing/reading the traid data set
titanic = pd.read_csv('train.csv')

In [426]:
#First 5 rows of train data
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [427]:
#Correlation between the variables
titanic.corr(method = 'pearson')

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [428]:
#Unique elements under Embarked column
titanic.Embarked.unique() 

array(['S', 'C', 'Q', nan], dtype=object)

In [429]:
titanic['Embarked'].isnull().sum()

2

In [430]:
#Missing values under Embarked
titanic[titanic['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


### Missing Data imputation

In [431]:
titanic_copy = titanic

In [432]:
#To convert the columns to numbers using replace
cleanup_nums = {"Sex":     {"female": 0, "male": 1},
                "Embarked": {"C": 1, "Q": 2, "S": 3 }}

titanic_copy = titanic_copy.replace(cleanup_nums)
titanic_copy.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3.0


In [433]:
titanic_copy[titanic_copy['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",0,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",0,62.0,0,0,113572,80.0,B28,NaN


In [434]:
titanic_copy.corr(method = 'pearson')

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.036847,-0.057527,-0.001652,0.012658,0.013166
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.077221,-0.035322,0.081629,0.257307,-0.169718
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.369226,0.083081,0.018443,-0.549500,0.164681
Sex,0.042939,-0.543351,0.131900,1.000000,0.093254,-0.114631,-0.245489,-0.182333,0.110320
Age,0.036847,-0.077221,-0.369226,0.093254,1.000000,-0.308247,-0.189119,0.096067,-0.032565
SibSp,-0.057527,-0.035322,0.083081,-0.114631,-0.308247,1.000000,0.414838,0.159651,0.068900
Parch,-0.001652,0.081629,0.018443,-0.245489,-0.189119,0.414838,1.000000,0.216225,0.040449
Fare,0.012658,0.257307,-0.549500,-0.182333,0.096067,0.159651,0.216225,1.000000,-0.226311
Embarked,0.013166,-0.169718,0.164681,0.110320,-0.032565,0.068900,0.040449,-0.226311,1.000000


In [435]:
#replacing the missing data with median of Embarked variable
titanic_copy['Embarked'].fillna(titanic_copy['Embarked'].median(), inplace=True)

In [436]:
#Missing data replaced rows
titanic_copy.loc[titanic_copy['Fare'] == 80]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",0,38.0,0,0,113572,80.0,B28,3.0
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",0,62.0,0,0,113572,80.0,B28,3.0


In [437]:
train_data = titanic_copy

In [442]:
#Reading Test data
#To convert the columns to numbers using replace in test data
test_data = pd.read_csv('test.csv')
test_data.head(5)
test_data = test_data.replace(cleanup_nums)
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,3
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,2
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,3
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,3


In [444]:
#y = train_data["Survived"]

#features = ["Age", "SibSp", "Parch", "Fare", "Embarked"]
#X = pd.get_dummies(train_data[features])
#X_test = pd.get_dummies(test_data[features])

In [445]:
#model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
#model.fit(X, y)
#predictions = model.predict(X_test)


In [447]:
train_data[train_data['Age'].isnull()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",1,NaN,0,0,330877,8.4583,NaN,2.0
17,18,1,2,"Williams, Mr. Charles Eugene",1,NaN,0,0,244373,13.0000,NaN,3.0
19,20,1,3,"Masselmani, Mrs. Fatima",0,NaN,0,0,2649,7.2250,NaN,1.0
26,27,0,3,"Emir, Mr. Farred Chehab",1,NaN,0,0,2631,7.2250,NaN,1.0
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",0,NaN,0,0,330959,7.8792,NaN,2.0


In [448]:
#There are still lot many missing values in both test and train datasets
#Lets try to get them replaced with Missing data imputation
pd.set_option("display.max_rows", None, "display.max_columns", None)
#First lets try to merge the train and test data
combined=train_data.append(test_data)

warnings.filterwarnings(action='once')

In [449]:
print('Missing values Percentage: \n\n', round (combined.isnull().sum().sort_values(ascending=False)/len(combined)*100,1))

Missing values Percentage: 

 Cabin          77.5
Survived       31.9
Age            20.1
Fare            0.1
Ticket          0.0
SibSp           0.0
Sex             0.0
Pclass          0.0
PassengerId     0.0
Parch           0.0
Name            0.0
Embarked        0.0
dtype: float64


In [450]:
#Since Cabin has a very high percentage of 77.5 missing values,
#it is useless trying to fill that as it may create lot of noise in the model/prediction.
#Survived missing values are due to test data which we will be predicting.
#So the next variable to fill in is Age as it is also a key factor to predict the survival.
#A better way to do this is by grouping into categories are finding the mean of replace the missing value.
#Lets find the boy children as they will have 'Master' prefix in their name
display(combined[(combined.Age.isnull()) & (combined.Name.str.contains('Master'))])

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
65,NaN,NaN,1.0,15.2458,"Moubarek, Master. Gerios",1,66,3,1,1,1.0,2661
159,NaN,NaN,3.0,69.5500,"Sage, Master. Thomas Henry",2,160,3,1,8,0.0,CA. 2343
176,NaN,NaN,3.0,25.4667,"Lefebre, Master. Henry Forbes",1,177,3,1,3,0.0,4133
709,NaN,NaN,1.0,15.2458,"Moubarek, Master. Halim Gonios (""William George"")",1,710,3,1,1,1.0,2661
244,NaN,NaN,3.0,23.4500,"Johnston, Master. William Arthur Willie""""",2,1136,3,1,1,NaN,W./C. 6607
339,NaN,NaN,1.0,7.2292,"Betros, Master. Seman",0,1231,3,1,0,NaN,2622
344,NaN,NaN,3.0,14.5000,"van Billiard, Master. James William",1,1236,3,1,1,NaN,A/5. 851
417,NaN,NaN,1.0,22.3583,"Peter, Master. Michael J",1,1309,3,1,1,NaN,2668


In [451]:
#calculating average of above group of Male children
print(train_data[train_data.Name.str.contains('Master')]['Age'].mean())

4.574166666666667


In [453]:
# We can say that 5 is an optimal age for this group. 
# But we can also see a 0 in Parch column, which means the kid is not acomppanied with any parent.
# Hence we will just assume that the Child
# is little senior in age and cannot be 5. We will assign the max value of Master 
# which is around 14 for such cases.
test_data.loc[test_data.PassengerId==1231,'Age']=14

In [455]:
test_data.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked         int64
dtype: object

In [456]:
#Now lets extract the titles from name column
train_data['Title'], test_data['Title'] = [df.Name.str.extract (' ([A-Za-z]+)\.', expand=False) for df in [train_data, test_data]]
warnings.filterwarnings(action='once')

<>:2: DeprecationWarning: invalid escape sequence \.


In [458]:
#Lets see the different title categories
train_data.groupby(['Title', 'Pclass'])['Age'].agg(['mean', 'count'])

mean  count
Title    Pclass                  
Capt     1       70.000000      1
Col      1       58.000000      2
Countess 1       33.000000      1
Don      1       40.000000      1
Dr       1       43.750000      4
         2       38.500000      2
Jonkheer 1       38.000000      1
Lady     1       48.000000      1
Major    1       48.500000      2
Master   1        5.306667      3
         2        2.258889      9
         3        5.350833     24
Miss     1       30.000000     45
         2       22.390625     32
         3       16.123188     69
Mlle     1       24.000000      2
Mme      1       24.000000      1
Mr       1       41.580460     87
         2       32.768293     82
         3       28.724891    229
Mrs      1       40.882353     34
         2       33.682927     41
         3       33.515152     33
Ms       2       28.000000      1
Rev      2       43.166667      6
Sir      1       49.000000      1

In [459]:
#Lets try to reduce the titles by categorising least used titles 
TitleDict = {"Capt": "Officer","Col": "Officer","Major": "Officer","Jonkheer": "Royalty", \
             "Don": "Royalty", "Sir" : "Royalty","Dr": "Royalty","Rev": "Royalty", \
             "Countess":"Royalty", "Mme": "Mrs", "Mlle": "Miss", "Ms": "Mrs","Mr" : "Mr", \
             "Mrs" : "Mrs","Miss" : "Miss","Master" : "Master","Lady" : "Royalty"}

In [460]:
train_data['Title'], test_data['Title'] = [df.Title.map(TitleDict) for df in [train_data, test_data]]

##Let us now reprint the groups
train_data.groupby(['Title', 'Pclass'])['Age'].agg(['mean', 'count'])

mean  count
Title   Pclass                  
Master  1        5.306667      3
        2        2.258889      9
        3        5.350833     24
Miss    1       29.744681     47
        2       22.390625     32
        3       16.123188     69
Mr      1       41.580460     87
        2       32.768293     82
        3       28.724891    229
Mrs     1       40.400000     35
        2       33.547619     42
        3       33.515152     33
Officer 1       56.600000      5
Royalty 1       42.555556      9
        2       42.000000      8

In [461]:
#Lets check both datasets
combined2 = train_data.append(test_data)
#train_data[train_data.Title.isnull()]
#test_data[test_data.Title.isnull()]

test_data.at[414,'Title'] = 'Royalty'
combined2.isnull().sum()

C:\Users\hp\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Age             262
Cabin          1014
Embarked          0
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
Title             1
dtype: int64

In [462]:
#Lets find female children now. As all unmarried females are titled miss, lets 
#consider that if Parch flag is >0 then they are most likely female children.
print ("Avg age of 'Miss' Title", round(train_data[train_data.Title=="Miss"]['Age'].mean()))

print ("Avg age of 'Miss' Title travelling without Parents", round(train_data[(train_data.Title=="Miss") & (train_data.Parch==0)]['Age'].mean()))

print ("Avg age of 'Miss' Title travelling with Parents", round(train_data[(train_data.Title=="Miss") & (train_data.Parch!=0)]['Age'].mean()), '\n')

Avg age of 'Miss' Title 22
Avg age of 'Miss' Title travelling without Parents 28
Avg age of 'Miss' Title travelling with Parents 12 



In [463]:
##Let us turn our attention to the missing fare
display(combined2[combined2.Fare.isnull()])

##Let us get fare per person
for df in [train_data, test_data, combined2]:
    df['PeopleInTicket']=df['Ticket'].map(combined2['Ticket'].value_counts())
    df['FarePerPerson']=df['Fare']/df['PeopleInTicket']


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title
152,60.5,NaN,3.0,NaN,"Storey, Mr. Thomas",0,1044,3,1,0,NaN,3701,Mr


In [464]:
##Just take the mean fare for the PORT S and the Pclass & fill it. Remember to consider FarePerPerson and not Fare
print('Mean fare for this category: ', train_data[(train_data.Embarked==3) & (train_data.Pclass==3)]['FarePerPerson'].mean())

Mean fare for this category:  7.382647773383372


In [465]:
#Filling the missing data
display(test_data[test_data.PassengerId == 1044])
#test_data.loc[test_data.PassengerId] = 1044

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,PeopleInTicket,FarePerPerson
152,1044,3,"Storey, Mr. Thomas",1,60.5,0,0,3701,NaN,NaN,3,Mr,1,NaN


In [466]:
#Filling the missing data
test_data.loc[test_data.PassengerId==1044,'Fare']=7.3826
display(test_data[test_data.PassengerId == 1044])
#test_data[test_data.PassengerId == 1044].dtypes

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,PeopleInTicket,FarePerPerson
152,1044,3,"Storey, Mr. Thomas",1,60.5,0,0,3701,7.3826,NaN,3,Mr,1,NaN


In [467]:
test_data.shape

(418, 14)

In [468]:
combined3 = train_data.append(test_data)
combined3  = combined3.drop(columns=['FarePerPerson'], axis=1)
combined3.tail()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,PeopleInTicket,Sex,SibSp,Survived,Ticket,Title
413,NaN,NaN,3.0,8.0500,"Spector, Mr. Woolf",0,1305,3,1,1,0,NaN,A.5. 3236,Mr
414,39.0,C105,1.0,108.9000,"Oliva y Ocana, Dona. Fermina",0,1306,1,3,0,0,NaN,PC 17758,Royalty
415,38.5,NaN,3.0,7.2500,"Saether, Mr. Simon Sivertsen",0,1307,3,1,1,0,NaN,SOTON/O.Q. 3101262,Mr
416,NaN,NaN,3.0,8.0500,"Ware, Mr. Frederick",0,1308,3,1,1,0,NaN,359309,Mr
417,NaN,NaN,1.0,22.3583,"Peter, Master. Michael J",1,1309,3,3,1,1,NaN,2668,Master


In [469]:
combined3.isnull().sum()

Age                262
Cabin             1014
Embarked             0
Fare                 0
Name                 0
Parch                0
PassengerId          0
Pclass               0
PeopleInTicket       0
Sex                  0
SibSp                0
Survived           418
Ticket               0
Title                0
dtype: int64

In [470]:
#Let us now come back to age filling. 
#Lets create a new Title entry for female children catgory 
for df in [train_data, test_data, combined3]:
    df.loc[(df.Title=='Miss') & (df.Parch!=0) & (df.PeopleInTicket>1), 'Title']="FemaleChild"

display(combined3[(combined3.Age.isnull()) & (combined3.Title=='FemaleChild')])

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,PeopleInTicket,Sex,SibSp,Survived,Ticket,Title
128,NaN,F E69,1.0,22.3583,"Peter, Miss. Anna",1,129,3,3,0,1,1.0,2668,FemaleChild
180,NaN,NaN,3.0,69.5500,"Sage, Miss. Constance Gladys",2,181,3,11,0,8,0.0,CA. 2343,FemaleChild
229,NaN,NaN,3.0,25.4667,"Lefebre, Miss. Mathilde",1,230,3,5,0,3,0.0,4133,FemaleChild
409,NaN,NaN,3.0,25.4667,"Lefebre, Miss. Ida",1,410,3,5,0,3,0.0,4133,FemaleChild
485,NaN,NaN,3.0,25.4667,"Lefebre, Miss. Jeannie",1,486,3,5,0,3,0.0,4133,FemaleChild
792,NaN,NaN,3.0,69.5500,"Sage, Miss. Stella Anna",2,793,3,11,0,8,0.0,CA. 2343,FemaleChild
863,NaN,NaN,3.0,69.5500,"Sage, Miss. Dorothy Edith ""Dolly""",2,864,3,11,0,8,0.0,CA. 2343,FemaleChild
888,NaN,NaN,3.0,23.4500,"Johnston, Miss. Catherine Helen ""Carrie""",2,889,3,4,0,1,0.0,W./C. 6607,FemaleChild
188,NaN,NaN,3.0,69.5500,"Sage, Miss. Ada",2,1080,3,11,0,8,NaN,CA. 2343,FemaleChild


In [471]:
#Now lets fill the missing age based on the mean of each category

## First lets define a group containing all the parameters you want and do a mean
## This will be our lookup table
grp = train_data.groupby(['Pclass','Sex','Title'])['Age'].mean()
print(grp)

Pclass  Sex  Title      
1       0    FemaleChild    21.071429
             Miss           33.424242
             Mrs            40.400000
             Royalty        43.333333
        1    Master          5.306667
             Mr             41.580460
             Officer        56.600000
             Royalty        42.166667
2       0    FemaleChild     9.916667
             Miss           29.875000
             Mrs            33.547619
        1    Master          2.258889
             Mr             32.768293
             Royalty        42.000000
3       0    FemaleChild     6.500000
             Miss           21.590909
             Mrs            33.515152
        1    Master          5.350833
             Mr             28.724891
Name: Age, dtype: float64


In [472]:
##So let us convert this 'series' object into a 'dataframe' 
grp = train_data.groupby(['Pclass','Sex','Title'])['Age'].mean().reset_index()[[ 'Pclass','Sex', 'Title', 'Age']]

In [473]:
grp

,Pclass,Sex,Title,Age
0,1,0,FemaleChild,21.071429
1,1,0,Miss,33.424242
2,1,0,Mrs,40.400000
3,1,0,Royalty,43.333333
4,1,1,Master,5.306667
5,1,1,Mr,41.580460
6,1,1,Officer,56.600000
7,1,1,Royalty,42.166667
8,2,0,FemaleChild,9.916667
9,2,0,Miss,29.875000


In [474]:
#Example of lookup: Age of male from pclass 2 and title Master
grp[(grp.Pclass==1) & (grp.Sex==1) & (grp.Title=='Master')]['Age'].values[0]

5.306666666666667

In [475]:
##Define a function called fill_age. This will lookup the combination
##passed to it using above lookup table and return the value of the age associated
def fill_age(x):
    return grp[(grp.Pclass==x.Pclass) & (grp.Sex==x.Sex) & (grp.Title==x.Title)]['Age'].values[0]

In [476]:
train_data.isnull().sum()

PassengerId         0
Survived            0
Pclass              0
Name                0
Sex                 0
Age               177
SibSp               0
Parch               0
Ticket              0
Fare                0
Cabin             687
Embarked            0
Title               0
PeopleInTicket      0
FarePerPerson       0
dtype: int64

In [477]:
train_data.isnull().sum()

PassengerId         0
Survived            0
Pclass              0
Name                0
Sex                 0
Age               177
SibSp               0
Parch               0
Ticket              0
Fare                0
Cabin             687
Embarked            0
Title               0
PeopleInTicket      0
FarePerPerson       0
dtype: int64

In [479]:
#Lets fill both test and train's age values by using lookup function

train_data['Age'], test_data['Age'] = [df.apply(lambda x: fill_age(x) if np.isnan(x['Age']) else x['Age'], axis=1) for df in [train_data, test_data]]

In [482]:
train_data.isnull().sum()

PassengerId         0
Survived            0
Pclass              0
Name                0
Sex                 0
Age                 0
SibSp               0
Parch               0
Ticket              0
Fare                0
Cabin             687
Embarked            0
Title               0
PeopleInTicket      0
FarePerPerson       0
dtype: int64

In [483]:
train_data.isnull().sum()

PassengerId         0
Survived            0
Pclass              0
Name                0
Sex                 0
Age                 0
SibSp               0
Parch               0
Ticket              0
Fare                0
Cabin             687
Embarked            0
Title               0
PeopleInTicket      0
FarePerPerson       0
dtype: int64

In [484]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,PeopleInTicket,FarePerPerson
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3.0,Mr,1,7.25000
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1.0,Mrs,2,35.64165
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3.0,Miss,1,7.92500
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3.0,Mrs,2,26.55000
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3.0,Mr,1,8.05000


In [485]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,PeopleInTicket,FarePerPerson
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,2,Mr,1,7.82920
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,3,Mrs,1,7.00000
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,2,Mr,1,9.68750
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,3,Mr,1,8.66250
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,3,Mrs,2,6.14375


## Modeling

#### Using random forest

In [486]:
#Since we have 0 missing values of required variables. Lets go ahead with modelling
train_data.corr(method = 'pearson')

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,PeopleInTicket,FarePerPerson
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.038214,-0.057527,-0.001652,0.012658,0.013128,-0.006376,0.006161
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.063778,-0.035322,0.081629,0.257307,-0.167675,0.064962,0.288337
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.400365,0.083081,0.018443,-0.549500,0.162098,-0.039893,-0.762981
Sex,0.042939,-0.543351,0.131900,1.000000,0.116264,-0.114631,-0.245489,-0.182333,0.108262,-0.176548,-0.114429
Age,0.038214,-0.063778,-0.400365,0.116264,1.000000,-0.302064,-0.206454,0.108221,-0.023276,-0.263095,0.338360
SibSp,-0.057527,-0.035322,0.083081,-0.114631,-0.302064,1.000000,0.414838,0.159651,0.068230,0.727331,-0.073529
Parch,-0.001652,0.081629,0.018443,-0.245489,-0.206454,0.414838,1.000000,0.216225,0.039798,0.638361,-0.036442
Fare,0.012658,0.257307,-0.549500,-0.182333,0.108221,0.159651,0.216225,1.000000,-0.224719,0.436544,0.827073
Embarked,0.013128,-0.167675,0.162098,0.108262,-0.023276,0.068230,0.039798,-0.224719,1.000000,0.022171,-0.275759
PeopleInTicket,-0.006376,0.064962,-0.039893,-0.176548,-0.263095,0.727331,0.638361,0.436544,0.022171,1.000000,0.054931


In [487]:
train_data = train_data.drop(columns=['FarePerPerson','PeopleInTicket'], axis=1)

In [488]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked       float64
Title           object
dtype: object

In [489]:
train_data['Age'] = train_data['Age'].astype(float)


In [490]:
test_data.dtypes

PassengerId         int64
Pclass              int64
Name               object
Sex                 int64
Age               float64
SibSp               int64
Parch               int64
Ticket             object
Fare              float64
Cabin              object
Embarked            int64
Title              object
PeopleInTicket      int64
FarePerPerson     float64
dtype: object

In [491]:
test_data['Age'][10]

28.724890829694324

In [492]:
test_data['Age'] = test_data['Age'].astype(float)

In [493]:
#selecting the features
y = train_data["Survived"]

features = ["Pclass", "Sex", "Age","SibSp", "Parch", "Fare", "Embarked"]



In [494]:
test_data[features].dtypes

Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked      int64
dtype: object

In [495]:
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

In [496]:
#using the random forest model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

In [503]:

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission2.csv', index=False) 
print("Your submission was successfully saved!")


Your submission was successfully saved!


### I submitted this output to kaggle and got an accuracy of 78.468%